In [1]:
import pandas as pd
import numpy 

In [4]:
synthetic_households = pd.read_csv(r"C:\Users\cra\San Diego Association of Governments\SANDAG QA QC - Documents\Projects\2023\2023-034 PopulationSim Outputs QC\Data\synthetic_households_2022_01.csv")

# Download the data

In [5]:
mgra_based_data = pd.read_csv(r"C:\Users\cra\San Diego Association of Governments\SANDAG QA QC - Documents\Projects\2023\2023-034 PopulationSim Outputs QC\Data\mgra15_based_input_2022_01.csv")
synthetic_households = pd.read_csv(r"C:\Users\cra\San Diego Association of Governments\SANDAG QA QC - Documents\Projects\2023\2023-034 PopulationSim Outputs QC\Data\synthetic_households_2022_01.csv")
synthetic_persons = pd.read_csv(r"C:\Users\cra\San Diego Association of Governments\SANDAG QA QC - Documents\Projects\2023\2023-034 PopulationSim Outputs QC\Data\synthetic_persons_2022_01.csv")

# Check 1
Null value checks - all 

In [8]:
mgra_based_data.isna().sum()[mgra_based_data.isna().sum() > 0]

acres       16
landacre    16
dtype: int64

In [9]:
synthetic_households.isna().sum()[synthetic_households.isna().sum() > 0]

WIF    491460
VEH    116411
dtype: int64

In [10]:
synthetic_persons.isna().sum()[synthetic_persons.isna().sum() > 0]

Series([], dtype: int64)

# Check 2
Consistency in Household Population, Group Quarter Population, and No. Of non-GQ households between MGRA-based input, synthetic persons and synthetic households files 

In [ ]:
synthetic_persons_with_GQ = synthetic_persons.merge(synthetic_households[['household_id', 'GQ_type']], how='left', on='household_id')
synthetic_persons_with_GQ

In [44]:
# Synthetic Households Work 
sh_number_of_non_gq_households = len(synthetic_households[synthetic_households['GQ_type'] == 0])
sh_population_non_gq = sum(synthetic_households[synthetic_households['GQ_type'] == 0]['NP'])
sh_population_gq = sum(synthetic_households[synthetic_households['GQ_type'] != 0]['NP'])

In [45]:
# Synthetic Persons Work 
sp_number_of_non_gq_households = synthetic_persons_with_GQ[synthetic_persons_with_GQ['GQ_type'] == 0]['household_id'].nunique()
sp_population_non_gq = len(synthetic_persons_with_GQ[synthetic_persons_with_GQ['GQ_type'] == 0])
sp_population_gq = len(synthetic_persons_with_GQ[synthetic_persons_with_GQ['GQ_type'] != 0])
sp_population_gq

116411

In [179]:
# MGRA based information 
mb_number_of_non_gq_households = sum(mgra_based_data['hh'])
mb_population_non_gq = sum(mgra_based_data['hhp'])
mb_population_gq = sum(mgra_based_data['gq_civ'] + mgra_based_data['gq_mil'])

In [172]:
sp_hh_non_gq = synthetic_persons_with_GQ[synthetic_persons_with_GQ['GQ_type'] == 0][['mgra']]
sp_hh_non_gq['sp_pop_non_gq_pop'] = 1
sp_hh_non_gq = sp_hh_non_gq.groupby('mgra').sum().reset_index()
sp_hh_non_gq

,mgra,sp_pop_non_gq_pop
0,1,441
1,2,67
2,3,547
3,4,6
4,5,90
...,...,...
18465,24314,50
18466,24316,515
18467,24317,8
18468,24318,134


In [173]:
mb_non_gq_pop = mgra_based_data[['mgra', 'hhp']]
mb_non_gq_pop.columns = ['mgra', 'mb_non_gq_pop']
mb_non_gq_pop

,mgra,mb_non_gq_pop
0,1,440
1,2,68
2,3,549
3,4,5
4,5,90
...,...,...
24316,24317,5
24317,24318,136
24318,24319,0
24319,24320,0


In [184]:
output = mb_non_gq_pop[~mb_non_gq_pop['mgra'].isin(list(sp_hh_non_gq['mgra']))]
output[output['mb_non_gq_pop'] != 0]

,mgra,mb_non_gq_pop
3307,3308,2
5021,5022,2
6649,6650,2
9886,9887,3
11115,11116,4
13622,13623,2
13869,13870,1
15803,15804,2
15959,15960,1
16220,16221,2


In [180]:
sum(mgra_based_data['hhp'])

3170895

In [176]:
output = sp_hh_non_gq.merge(mb_non_gq_pop, how='left', on='mgra')
output['diff'] = output['sp_pop_non_gq_pop'] - output['mb_non_gq_pop']

output.to_excel("mgra_non_gq_pop_differences.xlsx", index=False)

In [59]:
# Checking Equivalency 
print(sh_number_of_non_gq_households == sp_number_of_non_gq_households == mb_number_of_non_gq_households)
print(sh_population_non_gq == sp_population_non_gq == mb_population_non_gq)
print(sh_population_gq == sp_population_gq == mb_population_gq)

True
False
True


In [178]:
mb_population_non_gq

3170895

In [177]:
sp_population_non_gq

3165993

In [157]:
mb_population_non_gq - sh_population_gq

3054484

# Check 4: Internal Consistency Checks

### Check 4a: Internal Consistency Check in aggregate values across geography levels for both persons and households outputs  

In [70]:
# Region level 
synthetic_households['NP'].sum() == len(synthetic_persons)

True

In [82]:
sh_mgra_np = synthetic_households[['mgra', 'NP']].groupby('mgra').sum().reset_index()
sh_mgra_np.columns = ['mgra', 'pop_households']
sh_mgra_np

,mgra,pop_households
0,1,441
1,2,129
2,3,547
3,4,6
4,5,90
...,...,...
18624,24314,50
18625,24316,515
18626,24317,8
18627,24318,134


In [83]:
sp_pop_count = synthetic_persons[['mgra']]
sp_pop_count['pop_persons'] = 1
sp_pop_count = sp_pop_count.groupby('mgra').sum().reset_index()
sp_pop_count

C:\Users\cra\AppData\Local\Temp\ipykernel_13036\2966902140.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sp_pop_count['pop_persons'] = 1


,mgra,pop_persons
0,1,441
1,2,129
2,3,547
3,4,6
4,5,90
...,...,...
18624,24314,50
18625,24316,515
18626,24317,8
18627,24318,134


In [86]:
# Check for differences at the MGRA level 
sum(sh_mgra_np['pop_households'] - sp_pop_count['pop_persons'])

0

### Check 4b: Compare total 'NP' in the households output with count of 'SPORDER' in the persons output

In [96]:
sp_mgra_sporder = synthetic_persons[['mgra', 'SPORDER']].groupby('mgra').count().reset_index()
sp_mgra_sporder.columns = ['mgra', 'sporder_count']
sp_mgra_sporder

,mgra,sporder_count
0,1,441
1,2,129
2,3,547
3,4,6
4,5,90
...,...,...
18624,24314,50
18625,24316,515
18626,24317,8
18627,24318,134


In [98]:
# Check for differences at the MGRA level 
sum(sh_mgra_np['pop_households'] - sp_mgra_sporder['sporder_count'])

0

### 4c: Compare number of households match between the households and persons output   

In [101]:
synthetic_persons['household_id'].nunique() == len(synthetic_households)

True

### 4d: Compare workers ('numworkers' and 'isWorker') from households and persons output
- We need to figure out what is the new equivalent of 'isWorker' in persons

In [148]:
sp_household_esr = synthetic_persons[['household_id', 'ESR']]

In [149]:
sh_household_wif = synthetic_households[['household_id', 'WIF']]

In [156]:
# These household IDs say that there are no workers in the family but somebody in the household has an ESR ID of [1,2,4] which means that they are a worker.  
# What is the definition os 'isworker'
output = sp_household_esr.merge(sh_household_wif, how='left', on='household_id')
output = output[output['WIF'] == 0]
output = output[~output['ESR'].isin([0,3,6])]

output

,household_id,ESR,WIF
2217,873,1.0,0.0
5444,2080,1.0,0.0
8775,3477,1.0,0.0
10124,4068,1.0,0.0
14812,5830,1.0,0.0
...,...,...,...
3165246,1160281,1.0,0.0
3165319,1160301,1.0,0.0
3165323,1160302,1.0,0.0
3165327,1160303,1.0,0.0


In [145]:
synthetic_persons[~synthetic_persons['ESR'].isin([0,6])]

,mgra,household_id,SPORDER,AGEP,SEX,ESR,COW,WKHP,SCHG,RAC1P,HISP,MIL,SCHL,OCCP,WKW,NAICS2,SOC2
0,57,1,1,64,2,1.0,2.0,15.0,0,9,23,4.0,21.0,2050.0,1.0,81,21
1,57,1,2,29,1,1.0,1.0,40.0,0,9,3,4.0,19.0,4700.0,1.0,44,41
4,57,3,1,56,2,1.0,6.0,30.0,0,1,2,4.0,1.0,4230.0,2.0,81,37
6,57,3,3,26,1,3.0,9.0,0.0,0,1,2,4.0,20.0,9920.0,0.0,99,99
7,57,4,1,56,2,1.0,6.0,30.0,0,1,2,4.0,1.0,4230.0,2.0,81,37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3282340,24280,1276820,1,22,1,1.0,5.0,50.0,0,1,1,2.0,16.0,6515.0,1.0,23,47
3282354,24280,1276834,1,41,1,3.0,1.0,0.0,0,3,7,4.0,1.0,6515.0,0.0,23,47
3282364,24280,1276844,1,26,2,1.0,1.0,15.0,0,1,1,4.0,17.0,4110.0,6.0,722,35
3282390,24280,1276870,1,49,1,3.0,1.0,42.0,0,1,1,4.0,18.0,5350.0,5.0,56,43


In [147]:
synthetic_households[synthetic_households['WIF'] == 3]

,household_id,mgra,NP,HHADJINC,HHT,WIF,HUPAC,VEH,BLD,GQ_type
7,8,57,4,307483.5,1.0,3.0,4.0,4.0,2.0,0.0
8,9,57,4,307483.5,1.0,3.0,4.0,4.0,2.0,0.0
45,46,100,3,80775.0,1.0,3.0,4.0,2.0,7.0,0.0
46,47,100,4,193860.0,3.0,3.0,4.0,4.0,6.0,0.0
47,48,100,4,193860.0,3.0,3.0,4.0,4.0,6.0,0.0
...,...,...,...,...,...,...,...,...,...,...
1160450,1160451,10222,3,212169.0,1.0,3.0,4.0,4.0,2.0,0.0
1160451,1160452,10222,3,212169.0,1.0,3.0,4.0,4.0,2.0,0.0
1160452,1160453,10222,7,225093.0,1.0,3.0,2.0,5.0,5.0,0.0
1160463,1160464,10222,5,160365.3,3.0,3.0,2.0,4.0,2.0,0.0


In [123]:
synthetic_persons[synthetic_persons['COW'] > 0]

,mgra,household_id,SPORDER,AGEP,SEX,ESR,COW,WKHP,SCHG,RAC1P,HISP,MIL,SCHL,OCCP,WKW,NAICS2,SOC2
0,57,1,1,64,2,1.0,2.0,15.0,0,9,23,4.0,21.0,2050.0,1.0,81,21
1,57,1,2,29,1,1.0,1.0,40.0,0,9,3,4.0,19.0,4700.0,1.0,44,41
4,57,3,1,56,2,1.0,6.0,30.0,0,1,2,4.0,1.0,4230.0,2.0,81,37
6,57,3,3,26,1,3.0,9.0,0.0,0,1,2,4.0,20.0,9920.0,0.0,99,99
7,57,4,1,56,2,1.0,6.0,30.0,0,1,2,4.0,1.0,4230.0,2.0,81,37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3282391,24280,1276871,1,34,1,6.0,3.0,40.0,0,2,1,4.0,19.0,3602.0,4.0,62,31
3282392,24280,1276872,1,65,1,6.0,6.0,40.0,0,1,1,4.0,18.0,440.0,1.0,81,11
3282394,24280,1276874,1,29,1,6.0,1.0,0.0,0,2,1,4.0,16.0,9645.0,0.0,45,53
3282400,24280,1276880,1,28,1,6.0,1.0,40.0,0,1,2,4.0,11.0,6260.0,5.0,23,47


In [126]:
len(synthetic_persons[synthetic_persons['COW'] == 0])

1497299

In [136]:
hp_cow = synthetic_persons[['household_id', 'COW']].groupby('household_id').sum().reset_index()
hp_cow

,household_id,COW
0,1,3.0
1,2,0.0
2,3,15.0
3,4,15.0
4,5,0.0
...,...,...
1276878,1276879,0.0
1276879,1276880,1.0
1276880,1276881,0.0
1276881,1276882,0.0


In [142]:
workers_comparison = hp_cow.merge(synthetic_households[['household_id', 'WIF']], how='left', on='household_id')
#workers_comparison['Diff'] = workers_comparison['COW'] - workers_comparison['WIF']


workers_comparison[((workers_comparison['COW'] != 0) & (workers_comparison['WIF'] == 0))]

,household_id,COW,WIF
110,111,1.0,0.0
119,120,4.0,0.0
128,129,5.0,0.0
129,130,5.0,0.0
154,155,1.0,0.0
...,...,...,...
1160300,1160301,10.0,0.0
1160301,1160302,10.0,0.0
1160302,1160303,10.0,0.0
1160319,1160320,1.0,0.0


In [132]:
synthetic_households

,household_id,mgra,NP,HHADJINC,HHT,WIF,HUPAC,VEH,BLD,GQ_type
0,1,57,3,87344.70,3.0,2.0,4.0,1.0,6.0,0.0
1,2,57,1,39633.60,6.0,NaN,4.0,1.0,3.0,0.0
2,3,57,3,35541.00,1.0,1.0,4.0,5.0,3.0,0.0
3,4,57,3,35541.00,1.0,1.0,4.0,5.0,3.0,0.0
4,5,57,2,56865.60,1.0,0.0,4.0,3.0,2.0,0.0
...,...,...,...,...,...,...,...,...,...,...
1276878,1276879,24280,1,11308.50,0.0,NaN,0.0,NaN,0.0,3.0
1276879,1276880,24280,1,12924.00,0.0,NaN,0.0,NaN,0.0,3.0
1276880,1276881,24280,1,16962.75,0.0,NaN,0.0,NaN,0.0,3.0
1276881,1276882,24280,1,0.00,0.0,NaN,0.0,NaN,0.0,3.0


In [106]:
synthetic_households['WIF'].sum()

1157046.0

In [125]:
synthetic_households.describe()

,household_id,mgra,NP,HHADJINC,HHT,WIF,HUPAC,VEH,BLD,GQ_type
count,1.276883e+06,1.276883e+06,1.276883e+06,1.276883e+06,1.276883e+06,785423.000000,1.276883e+06,1.160472e+06,1.276883e+06,1.276883e+06
mean,6.384420e+05,1.052548e+04,2.570638e+00,1.282591e+05,2.391479e+00,1.473150,3.061090e+00,2.028680e+00,3.418195e+00,1.836965e-01
std,3.686045e+05,6.441142e+03,1.656219e+00,1.353380e+05,2.047466e+00,0.874175,1.358012e+00,1.140779e+00,2.661107e+00,6.350324e-01
min,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,3.192215e+05,5.549000e+03,1.000000e+00,4.071060e+04,1.000000e+00,1.000000,2.000000e+00,1.000000e+00,2.000000e+00,0.000000e+00
50%,6.384420e+05,9.694000e+03,2.000000e+00,9.369900e+04,1.000000e+00,2.000000,4.000000e+00,2.000000e+00,2.000000e+00,0.000000e+00
75%,9.576625e+05,1.514800e+04,3.000000e+00,1.701660e+05,4.000000e+00,2.000000,4.000000e+00,3.000000e+00,5.000000e+00,0.000000e+00
max,1.276883e+06,2.432100e+04,1.600000e+01,1.709307e+06,7.000000e+00,3.000000,4.000000e+00,6.000000e+00,1.000000e+01,3.000000e+00


# Check 5

### Check 5a: Compare number of persons by GQ and non-GQ in the persons and households output match forecast numbers.

In [108]:
print(f"Non-GQ Population - Synthetic Households - {sh_population_non_gq}")
print(f"Non-GQ Population - Synthetic Persons - {sp_population_non_gq}")
print(f"GQ Population - Synthetic Households - {sh_population_gq}")
print(f"GQ Population - Synthetic Households - {sh_population_gq}")

Non-GQ Population: Synthetic Households - 3165993


In [ ]:
print(sh_population_non_gq == sp_population_non_gq == mb_population_non_gq)
print(sh_population_gq == sp_population_gq == mb_population_gq)

|